In [8]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
#from multiprocessing import Pool, Process
import os


In [20]:
def get_all_page(urls, pattern, url_base):

    if pattern in urls:
        return(urls)
    
    html = requests.get(pattern)
    soup = BeautifulSoup(html.content, "html.parser")
    last_page = soup.find("a", {"title" : "Dernière page"})["page"]

    iter = soup.find_all("div", {"class" : re.compile("m3-pagination print_hidden|print_hidden pagination")})
    for item in iter:
        lst = re.findall("url.*?;", item.find_next_sibling().text)
        nw_url = ""
        if len(lst) > 0:
            for item in lst:
                if re.search("querystring", item):
                    continue
                else:
                    item = re.sub("url =|url \+=|url", "", item)
                    item = item.replace(";", "").replace("\"", "").strip()
                    item = item.split(" + ")
                for part in item:
                    nw_url += part
            match = re.compile("\$\(this\)\.attr\(page\)")
            for i in range(1, int(last_page)+1):
                url = urljoin(url_base, re.sub(match,str(i), nw_url))
                if url not in urls:
                    urls.append(url)
                    
        else:
            for i in range(1, int(last_page)+1):
                urls.append(urljoin(url_base, re.sub("page=.", f"page={i}", pattern)))
    
    return(urls)

In [49]:
def getAllUrl(url, url_base, urls, errors, EXPORT_FOLDER, EXPORT_FILE, counter):
    
    page = requests.get(url)

    if page.status_code == 200:
        soup = BeautifulSoup(page.content, "html.parser")
        for anchor in enumerate(soup.find_all("a")):
            counter += 1
            if "href" in anchor.attrs:
                if anchor["href"] == "JavaScript:void(0);" and anchor["class"] == "lien-page":
                    try:
                        urls.extend(get_all_page(urls), url, url_base)
                    except:
                        continue
                absolute_url = urljoin(url_base, anchor["href"])
                if "download" not in absolute_url and absolute_url not in urls and anchor["href"] != "JavaScript:void(0);":
                    if re.search(url_base, absolute_url):
                        urls.append(absolute_url)
        # Export des pages html ?
                        print(anchor["href"])
                        print(absolute_url)
                        with open(f"{EXPORT_FOLDER}{counter}.html", "w", encoding="utf-8") as f:
                            f.write(page.text)
                        with open(EXPORT_FILE, "a", encoding="utf-8") as f:
                            f.write(absolute_url)
    else:
        print(" ", page.status_code)
        errors.append(url)

    return(urls)

In [34]:
def read_urls(EXPORT_FILE):
    with open(EXPORT_FILE, "r") as f:
        urls = f.read().split("\n") 
    print(len(urls))
    return(urls)

In [50]:

EXPORT_FILE = "M3urls3.txt"
EXPORT_FOLDER = "urls/"

if __name__ == "__main__":
    
    url_base = 'https://www.millenaire3.com'
    errors = []
    counter = 0
    # urls = get_all_page(urls = [], 
    #                     pattern="https://www.millenaire3.com/m3_search?page=1&sort_by=0&SearchText=&modeRecherche=1&titre=&auteurId=&auteurText=&100%25=&100%25_new_value=true&date_debut=01%2F01%2F1900&date_debut_mobile=&date_fin=31%2F12%2F2025&date_fin_mobile=&identifiers=&rubriques=",
    #                     url_base = url_base)
    # if EXPORT_FILE in os.listdir():
    #     urls = read_urls(EXPORT_FILE)
    # else:
    #     urls = [url_base]

    with open(EXPORT_FILE, "w", encoding="utf-8") as f:
        for url in urls:
            f.write("")

    i = 468 # On peut changer la valeur de départ
    while i <= len(urls)-1:
        if urls[i].replace("_", "/").replace("https:__", "").replace("?", "!interrogation!")+".html" in os.listdir():
            continue
        elif re.search("/partage_email/|/generated_pdf/", urls[i]):
            errors.append(urls[i])
            print(f"\r{i} / {len(urls)}, errors", end = "")
            continue
        else:
            getAllUrl(urls[i], url_base, urls, errors, EXPORT_FOLDER, EXPORT_FILE, counter)
            i += 1
            print(f"\r{i} / {len(urls)} : {urls[i-1]}", end = "")

/dossiers/2022/cycle-veille-m3-air-ameliorer-la-qualite-de-ce-que-l-on-respire-une-urgence-vitale/veille-m3-pollution-de-l-air-notre-connexion-a-la-nature-la-premiere-des-protections-face-aux-menaces-diffuses
https://www.millenaire3.com/dossiers/2022/cycle-veille-m3-air-ameliorer-la-qualite-de-ce-que-l-on-respire-une-urgence-vitale/veille-m3-pollution-de-l-air-notre-connexion-a-la-nature-la-premiere-des-protections-face-aux-menaces-diffuses
/dossiers/2022/cycle-veille-m3-air-ameliorer-la-qualite-de-ce-que-l-on-respire-une-urgence-vitale/veille-m3-de-l-air-!-comme-la-horde-du-contrevent-sentir-le-mouvement-en-vivant-a-contre-courant
https://www.millenaire3.com/dossiers/2022/cycle-veille-m3-air-ameliorer-la-qualite-de-ce-que-l-on-respire-une-urgence-vitale/veille-m3-de-l-air-!-comme-la-horde-du-contrevent-sentir-le-mouvement-en-vivant-a-contre-courant
/dossiers/2022/cycle-veille-m3-air-ameliorer-la-qualite-de-ce-que-l-on-respire-une-urgence-vitale/veille-m3-pour-une-meilleure-qualite-de-

OSError: [Errno 22] Invalid argument: 'urls/www.millenaire3.com_dossiers_entre-decentralisation-des-competences-et-proximite-des-services-quels-enjeux-pour-la-territorialisation_caroline-richemont-dans-la-notion-de-relation-de-service-il-y-a-l-idee-de-creer-des-liens-qui-seront-amenes-a-vivre-et-evoluer.html'

In [45]:
urls[468]

'https://www.millenaire3.com/dossiers/2022/cycle-veille-m3-air-ameliorer-la-qualite-de-ce-que-l-on-respire-une-urgence-vitale'

In [34]:
urls = read_urls(EXPORT_FILE)

12367


In [5]:
tmp = get_all_page(["https://www.millenaire3.com#"], 
             "https://www.millenaire3.com/toutes-les-videos",
             "https://www.millenaire3.com/")

errors = []

getAllUrl("https://www.millenaire3.com#", "https://www.millenaire3.com/", tmp, errors)

['https://www.millenaire3.com#',
 'https://www.millenaire3.com/toutes-les-videos?p=1',
 'https://www.millenaire3.com/toutes-les-videos?p=2',
 'https://www.millenaire3.com/toutes-les-videos?p=3',
 'https://www.millenaire3.com/toutes-les-videos?p=4',
 'https://www.millenaire3.com/toutes-les-videos?p=5',
 'https://www.millenaire3.com/toutes-les-videos?p=6',
 'https://www.millenaire3.com/toutes-les-videos?p=7',
 'https://www.millenaire3.com/toutes-les-videos?p=8',
 'https://www.millenaire3.com/toutes-les-videos?p=9',
 'https://www.millenaire3.com/toutes-les-videos?p=10',
 'https://www.millenaire3.com/toutes-les-videos?p=11',
 'https://www.millenaire3.com/toutes-les-videos?p=12',
 'https://www.millenaire3.com/',
 'https://www.millenaire3.com/#content',
 'https://www.millenaire3.com/#main-menu',
 'https://www.millenaire3.com/#block-search',
 'https://www.millenaire3.com/nos-flux-rss',
 'https://www.millenaire3.com/societe',
 'https://www.millenaire3.com/modes-d-action',
 'https://www.millena